In [2]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time
import pickle
from datetime import datetime

Importing plotly failed. Interactive plots will not work.


# Global Model

In [3]:
total_records = 1064950

db = MongoClient(os.environ["DB_HOST"], int(os.environ["DB_PORT"]))
collection = 'covid_county_formatted'
pipeline = [
    {
        "$sample": {"size": total_records//100}
    }
]
cursor = db.sustaindb[collection].aggregate(pipeline)
df_original = pd.DataFrame(list(cursor))

In [9]:
df = df_original
covid_features = ['cases', 'deaths']
unique_dates = df['date'].unique()
df_map = {}

for selected_feature in covid_features:
    df_s = df[['date', selected_feature]]
    sums = []
    for t in unique_dates:
        sum_ = df_s[df_s['date'] == t][selected_feature].sum()
        sums.append(sum_)
    df_sums = pd.DataFrame(list(zip(unique_dates, sums)), columns=['ds', 'y'])
#     df_means.index.name = 'ds'
    df_map[selected_feature] = df_sums
    
df_map['cases'].head()

,ds,y
0,2021-01-07,1757
1,2020-06-08,110
2,2020-10-25,265
3,2020-07-20,659
4,2020-12-27,6043


In [11]:
pickle.dump(df_map, open('pickles/covid/df_map.pkl', 'wb'))

In [12]:
# Start by loading pickled 'df_map'
df_map = pickle.load(open('pickles/covid/df_map.pkl', 'rb'))

In [13]:
# select one feature
df0 = df_map['cases']
df0.head()
df0.columns

Index(['ds', 'y'], dtype='object')

## Build Global Model

In [14]:
def predict(df_train):
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300)
    df_train_forecast = m.predict(df_train_future)

    return fit_m, df_train_future, df_train_forecast

fit_m, df_train, df_train_forecast = predict(df0)

Initial log joint probability = -3.1737
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        856.93   4.23956e-06       100.084      0.9923      0.9923      121   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       856.932   1.65885e-05       103.203   1.623e-07       0.001      192  LS failed, Hessian reset 
     136       856.934   3.36273e-08       100.163      0.2146      0.9771      225   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [15]:
global_model = fit_m

In [17]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

df0_cv = cross_validation(fit_m, initial= '100 days', period='100 days', horizon='100')
df0_p = performance_metrics(df0_cv)
df0_p.head()

INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


  0%|          | 0/3 [00:00<?, ?it/s]

Initial log joint probability = -4.06139
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      63       282.705    9.9467e-09        100.48       0.168       0.168       85   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
Initial log joint probability = -3.1626
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      74       612.982   6.55743e-09       100.557      0.2472      0.2472      103   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
Initial log joint probability = -3.1832
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       854.235   7.01577e-07       99.5517      0.5651      0.5651      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     131       854.235   3.46827e-08    

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,0 days 00:00:00.000000100,2.100069e+06,1449.161391,1001.159585,1.415313,0.587237,0.733411,0.666667


In [18]:
model = fit_m
print(f'seasonality_prior_scale: {model.seasonality_prior_scale}')
print(f'changepoint_prior_scale: {model.changepoint_prior_scale}')
print(f'uncertainty_samples: {model.uncertainty_samples}')
print(f'seasonlity_mode: {model.seasonality_mode}')
print(f'interval_width: {model.interval_width}')
print(f'growth: {model.growth}')

seasonality_prior_scale: 10.0
changepoint_prior_scale: 0.05
uncertainty_samples: 1000
seasonlity_mode: additive
interval_width: 0.8
growth: linear


## GridSearch on Global Model

In [19]:
import itertools
from dask.distributed import Client

client = Client('lattice-150:8786')    


def grid_search(df):
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    time1 = time.monotonic()
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial= '100 days', period='100 days', horizon='100', parallel="dask")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    print(f'Time: {time.monotonic() - time1}')
    
    optimal_params = tuning_results[tuning_results['rmse'] == min(tuning_results['rmse'])]
    rmse = optimal_params['rmse'].iloc[0]
    changepoint_prior_scale = optimal_params['changepoint_prior_scale'].iloc[0]
    seasonality_prior_scale = optimal_params['seasonality_prior_scale'].iloc[0]
    
    return rmse, changepoint_prior_scale, seasonality_prior_scale  

In [20]:
rmse, changepoint_prior_scale, seasonality_prior_scale = grid_search(df0)

# Optimal values
print(f'rmse: {rmse}')
print(f'changepoint_prior_scale: {changepoint_prior_scale}')
print(f'seasonality_prior_scale: {seasonality_prior_scale}')

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -3.1737
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      62       806.397   4.00199e-09       5030.43      0.1458      0.1458       77   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -3.1737
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       811.427   2.00898e-06       5000.87           1           1      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     105        811.44   1.41032e-06       4656.05   2.678e-10       0.001      172  LS failed, Hessian reset 
     122       811.445   9.36789e-09       4758.17      0.3158      0.3158      196   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance


INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


...

Initial log joint probability = -3.1737
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       848.372    0.00912565       28.9317           1           1      114   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       849.929    0.00427142       32.5392      0.7994      0.7994      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       850.735    0.00376087       33.0263       0.391           1      363   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     364       851.002   9.87863e-05       14.8234   8.335e-06       0.001      498  LS failed, Hessian reset 
     399       851.027    0.00120007       16.9904           1           1      538   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     419       851.033   9.45736e-05     

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 3 forecasts with cutoffs between 2020-08-18 23:59:59.999999900 and 2021-03-06 23:59:59.999999900


Initial log joint probability = -3.1737
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99         848.6    0.00297262       26.3962           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       850.048    0.00231131       13.3927           1           1      228   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       850.699    0.00867182       32.4437           1           1      351   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     311       850.807   0.000422346       31.7506   1.939e-05       0.001      403  LS failed, Hessian reset 
     399       851.027   0.000728181       9.46524       3.245      0.3245      509   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     458       851.046   1.31902e-05     

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>


Time: 206.46187630901113
rmse: 1365.082761731964
changepoint_prior_scale: 0.5
seasonality_prior_scale: 10.0


In [33]:
gis_joins = df_original['GISJOIN'].unique()
print(f'#GISJOINs: {len(gis_joins)}')

#GISJOINs: 2882


## Dictionary of GISJOIN --> DF

In [34]:
import dask
selected_feature = 'cases'
child_dfs = {}

def get_df_by_gis_join(gis_join):
    print(gis_join, end=' ')
    cursor = db.sustaindb[collection].aggregate([{"$match": {"GISJOIN": gis_join}}])
    df = pd.DataFrame(list(cursor))[['date', selected_feature]]
    df.columns = ['ds','y']
    return df

futures = []
for gis_join in gis_joins:
    try:
        child_dfs[gis_join] = get_df_by_gis_join(gis_join)
    except:
        print(f'Error on {gis_join}')

G5000030 G5501210 G2100750 G4100490 G4500150 G2900510 G3700710 G5101250 G4803370 G2400450 G2901810 G5400050 G2400150 G5400090 G2601570 G2902090 G4000190 G3900330 G3200070 G4900190 G0201500 G2800570 G3100190 G1800050 G4100250 G2300250 G3800590 G3100430 G2500010 G4802030 G4802550 G5107400 G2800910 G3100770 G1800510 G4802070 G4700990 G0101150 G1701690 G5106000 G4701890 G4201090 G3901010 G2000910 G1701950 G4601370 G2100710 G4200050 G2800830 G2600290 G3701990 G3200150 G5300570 G3701270 G2600070 G3900350 G2700210 G1700270 G4801170 G3000210 G2200910 G2300210 G5400910 G4600950 G2200710 G2100890 G2200470 G2900690 G5100250 G0100450 G2701490 G4701730 G2100230 G2200970 G4500030 G5400570 G3000770 G3101210 G3000910 G3900630 G2801250 G1701170 G3000710 G1901350 G5101950 G4601270 G4800770 G3300010 G4800030 G2600330 G3600250 G3400090 G3900870 G4803090 G4701630 G1700930 G2101330 G3900550 G5101130 G4800510 G2900810 G1302110 G3100670 G3500470 G3701830 G1901650 G4804710 G2400430 G4500590 G3800090 G1701890 G

G1901430 G5105900 G2000150 G2600450 G5100650 G5106830 G5501090 G3500310 G1701630 G1901590 G4100550 G2900490 G1200050 G2300170 G3701390 G4100310 G1900170 G1701270 G2600590 G3600070 G1701150 G3900170 G0100750 G2200730 G4802790 G5400130 G5106400 G4700410 G2700770 G2101870 G4802330 G2900130 G4802270 G4701350 G1700150 G3700490 G2100210 G1800970 G2700730 G2901850 G3900490 G4000590 G1801310 G2901690 G0400170 G0500890 G0101010 G1800930 G3000590 G1800170 G2001030 G3200190 G2902150 G3900750 G4700610 G4701530 G2801550 G4000110 G4100170 G5101430 G2901150 G1901710 G1302130 G1901310 G4801110 G3800130 G5500910 G4200970 G4900430 G1300150 G4802150 G4701230 G5600450 G4700630 G0800150 G5400750 G3101050 G2901290 G2700990 G1302210 G4000770 G4803430 G4000490 G5100030 G4100290 G1800290 G2600410 G4802950 G3000290 G1800830 G2101130 G2800650 G2600490 G2100790 G4700210 G2601010 G2600390 G4600630 G4200410 G1200010 G3700510 G1701030 G4200250 G5105300 G5501110 G4000070 G2002010 G5100050 G3100230 G2201150 G2001550 G

G3800670 G4600130 G3100310 G0100770 G5501070 G3700530 G5107500 G2900570 G4804810 G2100530 G2600510 G3101350 G5500950 G2900610 G2000490 G1801510 G1300210 G2500070 G2102250 G2400270 G5500370 G3801030 G0501370 G4100150 G0501110 G2600230 G0800330 G4803650 G4000830 G2200070 G0400050 G4803790 G3601150 G2100570 G4100510 G2101650 G2000670 G0201700 G5500050 G4900130 G2400470 G2101710 G2701010 G2000290 G3100990 G0200200 G3700850 G2800990 G3700770 G3600430 G0101190 G3701770 G2601070 G5501130 G0800190 G3800830 G2800550 G1801590 G3701750 G5300330 G4800310 G4601110 G1900990 G4804350 G3100630 G3901550 G2700330 G2000570 G1200090 G4200730 G4600210 G3300110 G2700750 G1200290 G4803190 G5400550 G2902010 G4701770 G3701090 G4000670 G3101670 G4600430 G5100510 G5300290 G2901710 G5100790 G2101270 G0801110 G3500290 G2000370 G1900070 G2800850 G3500090 G3901730 G4701130 G1701670 G1901210 G4000010 G5101270 G1301430 G0801010 G1801710 G3900470 G3000090 G4800350 G4100590 G4800970 G4802230 G2001290 G2001270 G4802430 G

G5501370 G4802730 G2901610 G4600310 G4801790 G1700670 G1300070 G1300370 G4700350 G5300310 G2900090 G1901930 G3100590 G2902110 G3000470 G1801030 G3700110 G1302710 G3601070 G1700730 G5100310 G1700070 G0101110 G5107000 G3800910 G2700930 G4500550 G2801350 G4100210 G3901610 G1900050 G3200290 G2601410 G4800710 G0100890 G1301170 G0500610 G0101270 G2800070 G1801390 G3800030 G0201880 G1200270 G2101950 G3000170 G5500230 G3900410 G4500250 G1200570 G4500490 G4701330 G3901590 G2000050 G4801690 G4802190 G4000170 G3700650 G1200730 G4200870 G2000390 G3100010 G4801930 G2600830 G5101730 G1900290 G1301610 G4700650 G4600510 G1901410 G5000190 G0800390 G0601050 G1701390 G1801690 G5501050 G0800510 G2600730 G4201150 G2600610 G3100870 G4600850 G1901050 G0100190 G4803710 G2001010 G3500070 G1300010 G5400030 G2800490 G5300230 G5400790 G2102370 G3901170 G5101170 G2000530 G4801370 G3700550 G1302890 G4600870 G4201170 G5600250 G2801270 G2001530 G2900670 G3400350 G3400330 G2601530 G5300490 G2000850 G4001290 G1301970 G

In [27]:
pickle.dump(child_dfs, open('pickles/covid/child_dfs.pkl', 'wb'))

In [35]:
class TLModel:    
    def __init__(self, gis_join, time, model):
        self.rmse = rmse
        self.gis_join = gis_join
        self.time = time
        self.model = model
        
    def __str__(self):
        return f'{self.gis_join}: TLModel(time={self.time})'
    
class NonTLModel:
    def __init__(self, gis_join, time):
        self.gis_join = gis_join
        self.time = time
        
    def __str__(self):
        return f'{self.gis_join}: NonTLModel(time={self.time})'

# Child Models (Non-TL)

In [36]:
import time

non_tl_models = {}
counter = 1100
for gis_join in list(child_dfs.keys())[:100]:
    time1 = time.monotonic()
    predict(child_dfs[gis_join])
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    non_tl_models[gis_join] = NonTLModel(gis_join, time2)
    counter += 1

Initial log joint probability = -7.58523
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       698.183   6.77796e-06       98.9018           1           1      129   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       698.191   5.22915e-05       91.2124    5.17e-07       0.001      179  LS failed, Hessian reset 
     156       698.194   7.83914e-09       100.763      0.2164      0.2164      237   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
1 - G5000030: 4.2993900801520795 seconds
Initial log joint probability = -5.69869
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       727.912   2.09887e-06        99.463           1           1      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     100       727.912   4.27019

16 - G2902090: 4.386342637008056 seconds
Initial log joint probability = -3.95016
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      71       779.905   2.66543e-05        99.977   2.634e-07       0.001      128  LS failed, Hessian reset 
      94       779.908   5.66569e-09        101.07      0.1214      0.1214      160   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
17 - G4000190: 4.44537535100244 seconds
Initial log joint probability = -11.5151
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      93       638.468    8.3532e-05       101.906   8.147e-07       0.001      158  LS failed, Hessian reset 
      99       638.474   1.00394e-05       99.8791      0.5769      0.5769      164   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     153       638.495   2.82494e-08       101.288      0.30

34 - G3100770: 4.39449835778214 seconds
Initial log joint probability = -11.4212
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      89       660.928   4.82441e-07       102.805   4.524e-09       0.001      153  LS failed, Hessian reset 
      99       660.928   3.07782e-08       97.3162      0.4116           1      166   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     102       660.928   2.98019e-08       84.8771      0.5698      0.5698      169   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
35 - G1800510: 4.341004014015198 seconds
Initial log joint probability = -2.65176
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      65       687.456   1.48187e-08       104.159      0.3005           1       83   
Optimization terminated normally: 
  Convergence detected: relative gradient magnit

52 - G3200150: 4.389165328815579 seconds
Initial log joint probability = -3.43913
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      97       827.814   8.39166e-06       104.665   8.027e-08       0.001      198  LS failed, Hessian reset 
      99       827.814   7.47719e-06       104.367           1           1      200   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     154       827.822   2.00251e-08       93.0911      0.2825           1      269   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
53 - G5300570: 4.15404160390608 seconds
Initial log joint probability = -3.94271
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      83       811.471   1.04905e-05       99.8162   1.036e-07       0.001      146  LS failed, Hessian reset 
      99       811.472   1.83011e-07       99.2204      0.3

66 - G2100890: 4.362809994025156 seconds
Initial log joint probability = -6.12012
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      62         556.4   2.60101e-08         96.19      0.2879           1       84   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
67 - G2200470: 3.9751546517945826 seconds
Initial log joint probability = -4.56044
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      79       711.279   1.31491e-08       99.5743      0.2902           1      104   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
68 - G2900690: 4.288648869143799 seconds
Initial log joint probability = -4.64126
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      71       645.823   9.30779e-09        102.96       0.336       0.336       93   
Opti

86 - G4601270: 4.324560853186995 seconds
Initial log joint probability = -6.25133
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      67       568.102   6.05395e-09       98.5522      0.2204           1       92   
Optimization terminated normally: 
  Convergence detected: absolute parameter change was below tolerance
87 - G4800770: 4.290536785963923 seconds
Initial log joint probability = -10.4436
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      73       646.313   2.27737e-08         97.49      0.2887           1       98   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
88 - G3300010: 4.164143719011918 seconds
Initial log joint probability = -11.5699
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      65       394.693   8.69869e-09       100.395       0.459           1       92   
Optimi

# Child Models (TL)

In [40]:
def predict_transfer(df_train):
    # initilaize model with hyperparameters from parent model
    m = Prophet(
        seasonality_prior_scale = 10.0,
        changepoint_prior_scale = 0.5,
    )
    m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)

    return m, df_train_future, df_train_forecast

tl_models = {}
counter = 1
for gis_join in list(child_dfs.keys())[:100]:
    time1 = time.monotonic()
    m, df_train_future, df_train_forecast = predict_transfer(child_dfs[gis_join])
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    tl_models[gis_join] = TLModel(gis_join, time2, m)
    counter += 1

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -7.58523
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       604.422    0.00812085       12.4579      0.7555      0.7555      116   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       608.941    0.00245301        15.521      0.8901      0.8901      233   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299         610.9   0.000696861       8.57119      0.4859      0.4859      352   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       611.336    0.00355433       7.98961      0.2254           1      471   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       611.671    0.00109395       9.63771      0.5894      0.5894      594   
    Iter      log prob        ||dx||      ||grad||       alpha  

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 - G5000030: 2.3515129717998207 seconds
Initial log joint probability = -5.69869
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       670.203     0.0312899       42.4694           1           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       682.328    0.00419962       18.7927       3.171      0.3171      242   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       685.138     0.0369343       34.4862           1           1      363   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       692.313    0.00338411       40.5912           1           1      482   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     431       693.837   0.000136389        14.574   7.873e-06       0.001      554  LS failed, Hessian reset 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

97 - G2101330: 2.08098993008025 seconds
Initial log joint probability = -6.88748
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       722.465      0.014655       58.8905           1           1      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       730.625    0.00677918       24.9227      0.3401      0.3401      239   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       732.777    0.00255321       25.6554      0.3808      0.3808      367   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     379       733.655   0.000311587       29.2991   1.536e-05       0.001      508  LS failed, Hessian reset 
     392       733.831   9.16145e-05       9.78976   6.198e-06       0.001      559  LS failed, Hessian reset 
     399       733.883   0.000804494       7.39571           1      

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


98 - G3900550: 2.2711215389426798 seconds
Initial log joint probability = -9.27216
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       482.847    0.00578454       27.2091      0.1756       0.652      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199         485.5     0.0469767        10.643      0.5007           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     239       486.294   0.000377679       10.0056   2.797e-05       0.001      323  LS failed, Hessian reset 
     299       486.805   0.000522545       5.85138           1           1      398   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     338       486.865    0.00129107       14.0777   0.0001714       0.001      524  LS failed, Hessian reset 
     397       486.902    2.5492e-08       7.90293      0.1239    

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


99 - G5101130: 2.011434368090704 seconds
Initial log joint probability = -2.54864
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       831.667   0.000119312       8.13975      0.5041      0.5041      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     164       831.668   1.16944e-06       6.15779      0.2513           1      208   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
100 - G4800510: 1.9357968079857528 seconds


## Collect Times taken to build Models

In [31]:
import pandas as pd

non_tl_times = [x.time for x in non_tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(non_tl_models.keys(), non_tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('covid_non_tl_times.csv', index=False)

In [41]:
tl_times = [x.time for x in tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(tl_models.keys(), tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('covid_tl_times.csv', index=False)